In [2]:
import numpy as np
import pandas as pd
import gdreg

### Check .bed.annot.gz

In [21]:
FILE1 = "/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/ukb_imp_chr21_v3.bed.annot.gz"
FILE2 = "/n/groups/price/martin/data_GDREG/baseline_wes/bed_from_baselineLD_v2.2/bed_combined_WES_50K_imp_maf1en4/imp.chr21.minimac4.annot.gz"

In [24]:
temp_df1 = pd.read_csv(FILE1, sep='\t')
print('temp_df1', temp_df1.shape)
temp_df1.drop_duplicates(subset=['BP'], inplace=True)
temp_df1.index = temp_df1['BP']

temp_df2 = pd.read_csv(FILE2, sep='\t')
print('temp_df2', temp_df2.shape)
temp_df2.drop_duplicates(subset=['BP'], inplace=True)
temp_df2.index = temp_df2['BP']
temp_df2.columns = ["AN:%s"%x for x in temp_df2]

overlap_set = sorted(set(temp_df1.index) & set(temp_df2.index)) 
col_list = sorted(set(temp_df1.columns) & set(temp_df2.columns)) 
print('n_snp1=%d, n_snp2=%d, n_overlap=%d' % (temp_df1.shape[0], temp_df2.shape[0], len(overlap_set)))
print('n_col1=%d, n_col2=%d, n_overlap=%d' % (temp_df1.shape[1], temp_df2.shape[1], len(col_list)))

temp_df1 = temp_df1.loc[overlap_set, col_list]
temp_df2 = temp_df2.loc[overlap_set, col_list]

for col in col_list:
    if col.startswith("AN:"):
        print(col)
        print("    abs_dif=%0.4g, annot_size=%0.4g" % (
            np.absolute(temp_df1[col] - temp_df2[col]).sum(), 
            np.absolute(temp_df2[col]).sum()
        ))

temp_df1 (200644, 66)
temp_df2 (18696, 18)
n_snp1=200542, n_snp2=18499, n_overlap=12397
n_col1=66, n_col2=18, n_overlap=12
AN:Backgrd_Selection_Stat
    abs_dif=2.309e-14, annot_size=1579
AN:BivFlnk
    abs_dif=0, annot_size=375
AN:BivFlnk.extend.500
    abs_dif=0, annot_size=810
AN:Conserved_LindbladToh
    abs_dif=0, annot_size=1296
AN:Conserved_LindbladToh.extend.500
    abs_dif=0, annot_size=7174
AN:Conserved_Mammal_phastCons46way
    abs_dif=0, annot_size=786
AN:Conserved_Mammal_phastCons46way.extend.500
    abs_dif=0, annot_size=6053
AN:Conserved_Primate_phastCons46way
    abs_dif=0, annot_size=728
AN:Conserved_Primate_phastCons46way.extend.500
    abs_dif=0, annot_size=3628
AN:Conserved_Vertebrate_phastCons46way
    abs_dif=0, annot_size=870
AN:Conserved_Vertebrate_phastCons46way.extend.500
    abs_dif=0, annot_size=6613
AN:alleleage
    abs_dif=2.999e-11, annot_size=1.501e+08


### Create .bim from .pvar files

In [7]:
PGEN_FILE = "/n/scratch3/users/j/jz286/imp_geno/ukb_imp_chr@_v3"
for CHR in range(1,23):
    temp_df = gdreg.util.read_pgen(PGEN_FILE.replace("@", "%d"%CHR))["pvar"]
    df_bim = pd.DataFrame(data={
        "CHR" : temp_df["CHR"],
        "SNP" : temp_df["SNP"],
        "CM" : 0,
        "BP" : temp_df["BP"],
        "A1" : temp_df["REF"],
        "A2" : temp_df["ALT"],
    })
    df_bim.to_csv(
        "/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/bim/ukb_imp_chr%d_v3.bim" % CHR,
        sep=" ", header=False, index=False
    )